In [1]:
import json
from typing import List

from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, ConfigDict, Field

from sns_ai_automation_agency.agent.access import survey_access_information
from sns_ai_automation_agency.agent.restaurant import survey_restaurant_information

In [ ]:
thread_id = "test"
station_name = "横浜"
num_highlight_stations = 3
num_iterations = 2

access_data = survey_access_information(
    station_name=station_name,
    num_highlight_stations=num_highlight_stations,
    thread_id=thread_id,
)

restaurant_data = survey_restaurant_information(
    station_name=station_name,
    num_iterations=num_iterations,
    thread_id=thread_id,
)

In [3]:
access_json = json.dumps(access_data, indent=2, ensure_ascii=False).replace("{", "{{").replace("}", "}}")
restaurant_json = json.dumps(restaurant_data, indent=2, ensure_ascii=False).replace("{", "{{").replace("}", "}}")

In [14]:
system_prompt = """
あなたは住みたい駅の魅力を伝えるSNSショート動画の構成作家です。  
これから渡す JSON データを解析し、  動画全体の長さ（total_seconds）に合わせて
**データ内容に応じて最適なシーン構成（シーン数は可変）** を自動生成してください。

各シーンごとに  
「タイトル・内容・テロップ・画像検索クエリ・各シーンの開始/終了秒数」を出力します。

---

# 🎯【生成の目的】
- 駅の「住みやすさ」「街としての魅力」を直感的に伝える
- **店舗単位ではなく、エリア・ジャンル・雰囲気を中心に紹介する**
- シーンの数はデータ量に応じて 6〜14 で柔軟に生成
- SNS向けにテンポよく、過度な専門性や冗長表現は禁止

---

# 🧠【構成のポイント（重要）】

## ■ 店舗名について（必ず守る）
- **店舗名は原則禁止**
- 店舗情報は **エリアの特徴として抽象化** して表現する  
  例：  
  - 「山が見えるテラスカフェが多いエリア」  
  - 「ラーメン屋が並ぶ路地」  
  - 「レトロな町中華が残る飲み歩きエリア」
- 施設名（大型商業施設・公園・商店街など）は使用してOK  

→ **個別店舗NG / エリア名・施設名OK / ジャンルOK**

---

# 🧩【出力仕様】


構造体の形式で出力してください: 

**Scene X**
- タイトル：
- 内容：（1〜2文、事実ベース）
- テロップ：
- 画像検索クエリ：
- シーン開始秒数：（小数第一位）
- シーン終了秒数：（小数第一位）

---

# ⚙️【生成ルール】

## ■ Scene1〜3（構造固定）
1. 導入（抽象キャッチ.駅名は入れないこと）  
2. 駅名（テロップに必ず駅名を入れること）
3. 駅全体の印象（街の雰囲気 × アクセス × エリア型の魅力）

## ■ Scene4 以降はデータ依存で動的生成
- アクセス：1〜3  
- 飲食：1〜3（店舗名NG。ジャンル/エリア/雰囲気で説明）  
- 買い物：0〜2  
- 公園：0〜2  
- 商店街/景観：任意  
- ファミリー：任意  
- まとめ：1  
- 締め：1  

---

# 🎨【テロップのルール強化】

■ 導入・景観・締め → 感情に寄せる（8〜14文字）  
例：  
- 「昼も夜も絵になる街」  
- 「海と夜景の世界へ」

■ アクセス・飲食・買い物 → 情報性を入れる（12〜18文字）  
例：  
- 「新宿へ乗換なし約15分」  
- 「駅近にカフェとスイーツ充実」  
- 「公園が徒歩圏」

■ まとめ系 → 要素を凝縮（12〜18文字）  
例：  
- 「アクセス良好で遊びも充実」

■ 共通ルール
- ナレーションで読みやすいようにすること
NG例：「カフェx景色x夜景」のように掛け算表現はナレーションで読みにくいので禁止

---

#【シーン秒数の割り当て仕様】

## ■ 秒数配分の基本方針
1. シーン数はデータ量に応じて 6〜14 の範囲で自動決定する  
2. 各シーンに **start_sec** と **end_sec** を付与する  
3. 導入シーンは短めに（1〜3秒）  
4. 駅名・総評は 1〜3秒  
5. 中盤の情報シーン（アクセス/飲食/買い物/自然）は  
   全体の **60〜75%** を均等 or 比例配分  
6. 最後の「まとめ」「締め」は 2〜4秒  
7. 全シーンの秒数合計が total_seconds と一致するように調整する  
8. 端数は最終シーンで吸収してよい

---

# ❗注意（最重要）
- JSON に存在しない店舗名・固有名詞は書かない  
- 個別店舗を特定できる説明は避ける  
- 必ず「エリアとしての魅力」「ジャンル」「雰囲気」でまとめる

---

最適なシーン構成とテロップを生成してください。
"""

In [15]:
total_seconds = 15

user_prompt = f"""
以下の JSON データをもとに、  
上記のルールに従って、住みたい駅の魅力を伝えるSNS ショート動画のシーン構成を生成してください。

出力は「Scene 1 〜 最終シーン」まで、可変長です。

動画全体の秒数: {total_seconds}秒

---

【入力データ】

## 主要駅へのアクセス
{access_json}

## 駅周辺の飲食店情報
{restaurant_json}
"""

In [16]:
class Scene(BaseModel):
    """シーンの構造化レスポンス"""

    model_config = ConfigDict(from_attributes=True)

    title: str = Field(..., description="シーンのタイトル")
    content: str = Field(..., description="シーンの内容")
    telop: str = Field(..., description="シーンのテロップ")
    image_search_query: str = Field(..., description="画像検索クエリ")
    start_sec: float = Field(..., description="シーン開始秒数")
    end_sec: float = Field(..., description="シーン終了秒数")


class MovieResponse(BaseModel):
    """動画の構造化レスポンス"""

    model_config = ConfigDict(from_attributes=True)
    scenes: List[Scene] = Field(..., description="発見したシーンのリスト")

In [17]:
scene_llm = ChatOpenAI(model="gpt-5.1", temperature=0, reasoning_effort="high")
scene_llm = scene_llm.with_structured_output(MovieResponse)
scene_prompt = ChatPromptTemplate.from_messages([("system", system_prompt), ("user", user_prompt)])

chain = scene_prompt | scene_llm
scene_output = chain.invoke({})

In [18]:
for scene in scene_output.scenes:
    print(f"タイトル: {scene.title}")
    print(f"内容: {scene.content}")
    print(f"テロップ: {scene.telop}")
    print(f"画像検索クエリ: {scene.image_search_query}")
    print(f"開始秒数: {scene.start_sec}")
    print(f"終了秒数: {scene.end_sec}")
    print("-----")

タイトル: 導入：海と街が交わる日常
内容: 港の夜景と高層ビルが並ぶエリアに、昔ながらの飲み屋街も共存しています。都市と港町の空気が同時に味わえるロケーションです。
テロップ: 毎日が小旅行みたい
画像検索クエリ: 横浜 みなとみらい 高層ビル 港 夜景 住宅街 俯瞰
開始秒数: 0.0
終了秒数: 1.0
-----
タイトル: 駅名紹介：横浜駅エリア
内容: 今回は、神奈川の玄関口『横浜駅』周辺を、住む目線で紹介します。巨大ターミナルを中心に多彩な街が広がります。
テロップ: きょう歩く街は横浜駅周辺
画像検索クエリ: 横浜駅 駅前 ロータリー ビル群 ターミナル 昼
開始秒数: 1.0
終了秒数: 2.0
-----
タイトル: 全体像：ターミナル＋多エリア都市
内容: 複数のJRや地下鉄が集まるターミナルで、3分の桜木町や直通のみなとみらいなど雰囲気の違う街へすぐ行けます。通勤・買い物・レジャーが1駅圏にまとまった暮らしやすいポジションです。
テロップ: 海も都会もそろう街
画像検索クエリ: 横浜駅 周辺地図 みなとみらい 桜木町 野毛 エリア構成
開始秒数: 2.0
終了秒数: 3.5
-----
タイトル: アクセス：都心直通のターミナル力
内容: 東京駅へはJR東海道本線で乗換不要・約25分、新宿駅へもJR湘南新宿ライン快速で約31分です。品川駅は約12分、近隣の桜木町駅へもJR京浜東北線で約3分と、都心も横浜エリアも軽快に行き来できます。
テロップ: 東京25分・新宿31分直通
画像検索クエリ: JR横浜駅 ホーム 東海道線 湘南新宿ライン 電車 発着
開始秒数: 3.5
終了秒数: 5.9
-----
タイトル: 飲食①：みなとみらい＆東口カフェ
内容: みなとみらいは、海・観覧車・夜景に囲まれたモダンで開放的なカフェ＆レストランエリアです。横浜赤レンガ倉庫やランドマークタワー周辺の海辺プロムナードや、東口ポートサイドの窓が大きい店で、港を眺めながらランチやスイーツを楽しめます。
テロップ: 海沿いカフェと眺望ランチ充実
画像検索クエリ: みなとみらい 海沿いカフェ テラス席 観覧車 横浜赤レンガ倉庫 ランドマークタワー 昼
開始秒数: 5.9
終了秒数: 8.3
-----
タイトル: 飲食②：元町・中華街の食べ歩き
内容: 元町・中華街エリア